In [1]:
import numpy as np
import pandas as pd
import json
#import re
#import os
import torch
import transformers as ppb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
#import tensorflow as tf
#import tensorflow_hub as hub
#from tensorflow.keras import layers
#from transformers import TFDistilBertModel, DistilBertTokenizer
#from transformers.file_utils import is_tf_available
#from sklearn.model_selection import train_test_split

In [8]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [5]:
model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'activation_13', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [2]:
# Open json file
with open('C:/Users/plich/Downloads/NLP_AI/percent/cocaPercent10Ksample0.json') as infile:
    data = json.load(infile)
    
data_df = pd.DataFrame.from_dict(data, orient='columns')

data_df

,text,genre
5026391_5,to identify an individual digital file as a ki...,ACAD
4080783_15,Russia is looking for strategic alliances with...,ACAD
4080139_1,and many sections have been established ; toda...,ACAD
4005982_1,percent of the shares. 33 Buyer Selection <p> ...,ACAD
4019219_6,of clubfoot . A survey that was given to 541 p...,ACAD
...,...,...
5250896_1,"Mike was in is partly his own fault , but that...",WEB
5034603_1,'s basically it . I will come back if I ever h...,WEB
5059148_3,talking about a change from about 13.5 to 14.3...,WEB
5077786_1,5077786 <h> What are the most stolen vehicles ...,WEB


In [3]:
#Import pretrained model and tokenizer
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

#load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [4]:
#Tokenize the dataset
tokenized = data_df['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True)))

In [5]:
#Reduce sequence lengths over max and pad sequence lengths under max
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [6]:
np.array(padded).shape

(10815, 512)

In [7]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(10815, 512)

In [8]:
#torch.set_default_tensor_type('torch.cuda.FloatTensor')
dtype = torch.cuda.FloatTensor
input_ids = torch.tensor(padded).type(dtype)
attention_mask = torch.tensor(attention_mask).type(dtype)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask).cuda()

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx